In [92]:
import numpy as np
import pandas as pd

In [93]:
resaleflat1 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')
#resaleflat2 = pd.read_csv('resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')
resaleflat3 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv')
resaleflat4 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv')
resaleflat = pd.concat((resaleflat1,resaleflat3,resaleflat4), axis=0, sort=True)

resaleflat_l = resaleflat.copy()
resaleflatview = resaleflat1.copy()

In [94]:
resaleflat.head()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
0,406,Improved,2 ROOM,44.0,1979,2017-01,61 years 04 months,232000.0,10 TO 12,ANG MO KIO AVE 10,ANG MO KIO
1,108,New Generation,3 ROOM,67.0,1978,2017-01,60 years 07 months,250000.0,01 TO 03,ANG MO KIO AVE 4,ANG MO KIO
2,602,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,262000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO
3,465,New Generation,3 ROOM,68.0,1980,2017-01,62 years 01 month,265000.0,04 TO 06,ANG MO KIO AVE 10,ANG MO KIO
4,601,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,265000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO


In [95]:
resaleflat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139788 entries, 0 to 37152
Data columns (total 11 columns):
block                  139788 non-null object
flat_model             139788 non-null object
flat_type              139788 non-null object
floor_area_sqm         139788 non-null float64
lease_commence_date    139788 non-null int64
month                  139788 non-null object
remaining_lease        87585 non-null object
resale_price           139788 non-null float64
storey_range           139788 non-null object
street_name            139788 non-null object
town                   139788 non-null object
dtypes: float64(2), int64(1), object(8)
memory usage: 12.8+ MB


In [96]:
resaleflat.isnull().sum()

#Remaining lease has a lot of null, need to fix

block                      0
flat_model                 0
flat_type                  0
floor_area_sqm             0
lease_commence_date        0
month                      0
remaining_lease        52203
resale_price               0
storey_range               0
street_name                0
town                       0
dtype: int64

In [97]:
towndict = {}

for i, town in enumerate(resaleflat.town.unique()):
    towndict[town] = i
    
print(towndict)

{'ANG MO KIO': 0, 'BEDOK': 1, 'BISHAN': 2, 'BUKIT BATOK': 3, 'BUKIT MERAH': 4, 'BUKIT PANJANG': 5, 'BUKIT TIMAH': 6, 'CENTRAL AREA': 7, 'CHOA CHU KANG': 8, 'CLEMENTI': 9, 'GEYLANG': 10, 'HOUGANG': 11, 'JURONG EAST': 12, 'JURONG WEST': 13, 'KALLANG/WHAMPOA': 14, 'MARINE PARADE': 15, 'PASIR RIS': 16, 'PUNGGOL': 17, 'QUEENSTOWN': 18, 'SEMBAWANG': 19, 'SENGKANG': 20, 'SERANGOON': 21, 'TAMPINES': 22, 'TOA PAYOH': 23, 'WOODLANDS': 24, 'YISHUN': 25}


In [98]:
towndict = {}
flatdict = {}
blockdict = {}
streetnamedict = {}
storeydict = {}
flatmodeldict = {}
#Array that stores all dicts to change categorical into bins
alldicts = [towndict, flatdict, blockdict, streetnamedict, storeydict, flatmodeldict, flatmodeldict]
#column names that need to convert from category to value
colnames = ['town','flat_type','block','street_name','storey_range','flat_model']
print(colnames)

#As alldicts[n] is the dictionary for colnames[n], we can zip them up
for pair in zip(alldicts,colnames):
    #pair[1] is particular column name, get all unique terms in that column
    #save each value as a key with a number pair
    for i, value in enumerate(resaleflat[pair[1]].unique()):
        pair[0][value] = i

print(flatdict)

['town', 'flat_type', 'block', 'street_name', 'storey_range', 'flat_model']
{'2 ROOM': 0, '3 ROOM': 1, '4 ROOM': 2, '5 ROOM': 3, 'EXECUTIVE': 4, '1 ROOM': 5, 'MULTI-GENERATION': 6}


In [99]:
#Convert all categorial values into number bins

for pair in zip(alldicts,colnames):
    resaleflat[pair[1]] = resaleflat[pair[1]].replace(pair[0])
    
    
#ALTERNATIVELY, USE pd.get_dummies(dataframe)

In [100]:
#Convert first two char into integer for years left
resaleflat['remaining_lease_year'] = resaleflat['remaining_lease'].str.slice(stop=2).astype('float')
#Substring/slice the month out
resaleflat['remaining_lease_month'] = resaleflat['remaining_lease'].str.slice(start=9,stop=12)

In [101]:
resaleflat['remaining_lease_month'].unique()

array(['04 ', '07 ', '05 ', '01 ', '', '06 ', '08 ', '10 ', '03 ', '09 ',
       '11 ', '02 ', '0 m', nan], dtype=object)

In [102]:
replacemonth = {}
replacemonth[''] = 0
replacemonth['0 m'] = 0

resaleflat['remaining_lease_month'] = resaleflat['remaining_lease_month'].replace(replacemonth)

In [103]:
resaleflat['remaining_lease_month'] = resaleflat['remaining_lease_month'].astype('float') / 12

In [104]:
resaleflat['remaining_lease'] = resaleflat['remaining_lease_year'] + resaleflat['remaining_lease_month']
resaleflat = resaleflat.drop(['remaining_lease_year','remaining_lease_month'], axis=1)
resaleflat.head()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
0,0,0,0,44.0,1979,2017-01,61.333333,232000.0,0,0,0
1,1,1,1,67.0,1978,2017-01,60.583333,250000.0,1,1,0
2,2,1,1,67.0,1980,2017-01,62.416667,262000.0,1,2,0
3,3,1,1,68.0,1980,2017-01,62.083333,265000.0,2,0,0
4,4,1,1,67.0,1980,2017-01,62.416667,265000.0,1,2,0


In [105]:
resaleflat.month.unique()

array(['2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06',
       '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12',
       '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
       '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12',
       '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2012-03',
       '2012-04', '2012-05', '2012-06', '2012-07', '2012-08', '2012-09',
       '2012-10', '2012-11', '2012-12', '2013-01', '2013-02', '2013-03',
       '2013-04', '2013-05', '2013-06', '2013-07', '2013-08', '2013-09',
       '2013-10', '2013-11', '2013-12', '2014-01', '2014-02', '2014-03',
       '2014-04', '2014-05', '2014-06', '2014-07', '2014-08', '2014-09',
       '2014-10', '2014-11', '2014-12', '2015-01', '2015-02', '2015-03',
       '2015-04', '2015-05', '2015-06', '2015-07', '2015-08', '2015-09',
       '2015-10', '2015-11', '2015-12', '2016-01', '2016-02', '2016-03',
       '2016-04', '2016-05', '2016-06', '2016-07', 

In [106]:
resaleflat.month = resaleflat.month.apply(lambda x: float(x[0:4])+float(x[5:7])/12)

In [107]:
#Inpute missing values with remaining lease (take current year-month minus the lease start year)
resaleflat.remaining_lease[resaleflat.remaining_lease.isnull()] = resaleflat['month'] - resaleflat['lease_commence_date']

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [108]:
#Normalize the month (timeframe)
resaleflat.month = (resaleflat.month - resaleflat.month.mean()) / resaleflat.month.std()

In [109]:
resaleflat.month.describe()

count    1.397880e+05
mean    -4.665211e-11
std      1.000000e+00
min     -1.676358e+00
25%     -9.117660e-01
50%      4.397440e-02
75%      8.850260e-01
max      1.611389e+00
Name: month, dtype: float64

In [110]:
resaleflat.head()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
0,0,0,0,44.0,1979,0.540959,61.333333,232000.0,0,0,0
1,1,1,1,67.0,1978,0.540959,60.583333,250000.0,1,1,0
2,2,1,1,67.0,1980,0.540959,62.416667,262000.0,1,2,0
3,3,1,1,68.0,1980,0.540959,62.083333,265000.0,2,0,0
4,4,1,1,67.0,1980,0.540959,62.416667,265000.0,1,2,0


In [111]:
Corr = resaleflat.corr()
Corr

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
block,1.000000,0.160927,0.187160,0.152616,0.408713,0.050399,0.014057,0.098292,0.023593,0.178578,0.203929
flat_model,0.160927,1.000000,0.154035,0.214878,0.429213,0.051201,0.008593,0.162362,0.007205,0.130333,0.157540
flat_type,0.187160,0.154035,1.000000,0.937656,0.407706,0.025643,-0.045280,0.664892,0.032951,0.120852,0.138018
floor_area_sqm,0.152616,0.214878,0.937656,1.000000,0.363838,0.023323,-0.047429,0.667404,0.015810,0.123028,0.134292
lease_commence_date,0.408713,0.429213,0.407706,0.363838,1.000000,0.141427,0.068331,0.356885,0.082460,0.197763,0.221989
month,0.050399,0.051201,0.025643,0.023323,0.141427,1.000000,0.755874,-0.073186,-0.297650,0.019858,0.007296
remaining_lease,0.014057,0.008593,-0.045280,-0.047429,0.068331,0.755874,1.000000,-0.067100,-0.126129,0.016789,-0.004254
resale_price,0.098292,0.162362,0.664892,0.667404,0.356885,-0.073186,-0.067100,1.000000,0.153230,0.029097,-0.048014
storey_range,0.023593,0.007205,0.032951,0.015810,0.082460,-0.297650,-0.126129,0.153230,1.000000,-0.012344,-0.023875
street_name,0.178578,0.130333,0.120852,0.123028,0.197763,0.019858,0.016789,0.029097,-0.012344,1.000000,0.776911


In [112]:
#Disable correlation of 1 because its comparing with itself
#Without using enumerate and val1 > val2 (upper right of matrix), 32 sets of 16 pairs were printed (theres a duplicate for each)
#With val1 > val2, non duplicate sets are shown
#Iterate through column, then through index


for val1, column in enumerate(Corr):
    for val2, index in enumerate(Corr[column].index):
        if 0.2 <= Corr[column][index] < 1 and val1 > val2:
            print(str(Corr[column][index]) + ' ' + str(column) + ' and ' + str(index))

0.21487831914663114 floor_area_sqm and flat_model
0.9376558785948783 floor_area_sqm and flat_type
0.4087131536391544 lease_commence_date and block
0.42921304290920315 lease_commence_date and flat_model
0.407706107800434 lease_commence_date and flat_type
0.36383846026146727 lease_commence_date and floor_area_sqm
0.7558743253182897 remaining_lease and month
0.664891635819609 resale_price and flat_type
0.6674044393645352 resale_price and floor_area_sqm
0.35688514956690365 resale_price and lease_commence_date
0.2039289565530678 town and block
0.2219891653953732 town and lease_commence_date
0.7769110008144356 town and street_name


In [113]:
resaleflatview.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [114]:
#Show very bad correlations, corr > 0.4

for val1, column in enumerate(Corr):
    for val2, index in enumerate(Corr[column].index):
        if 0.4 <= Corr[column][index] < 1 and val1 > val2:
            print(str(Corr[column][index]) + ' ' + str(column) + ' and ' + str(index))

0.9376558785948783 floor_area_sqm and flat_type
0.4087131536391544 lease_commence_date and block
0.42921304290920315 lease_commence_date and flat_model
0.407706107800434 lease_commence_date and flat_type
0.7558743253182897 remaining_lease and month
0.664891635819609 resale_price and flat_type
0.6674044393645352 resale_price and floor_area_sqm
0.7769110008144356 town and street_name


In [115]:
resaleflat.tail()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
37148,1108,0,3,112.0,2000,0.50273,17.0,488000.0,4,473,25
37149,832,0,3,122.0,1987,0.50273,30.0,455000.0,1,362,25
37150,1402,4,4,181.0,1992,0.50273,25.0,778000.0,0,352,25
37151,1576,10,4,146.0,1988,0.50273,29.0,575000.0,1,361,25
37152,1084,17,6,164.0,1987,0.50273,30.0,735000.0,0,352,25


In [116]:
resaleflat.columns.values

array(['block', 'flat_model', 'flat_type', 'floor_area_sqm',
       'lease_commence_date', 'month', 'remaining_lease', 'resale_price',
       'storey_range', 'street_name', 'town'], dtype=object)

# We had to convert categorical to numerical first to see correlations, now we redo the table using onehotencoding/pd.getdummies

In [117]:
resaleflat_l.head()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
0,406,Improved,2 ROOM,44.0,1979,2017-01,61 years 04 months,232000.0,10 TO 12,ANG MO KIO AVE 10,ANG MO KIO
1,108,New Generation,3 ROOM,67.0,1978,2017-01,60 years 07 months,250000.0,01 TO 03,ANG MO KIO AVE 4,ANG MO KIO
2,602,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,262000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO
3,465,New Generation,3 ROOM,68.0,1980,2017-01,62 years 01 month,265000.0,04 TO 06,ANG MO KIO AVE 10,ANG MO KIO
4,601,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,265000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO


In [118]:
#Convert first two char into integer for years left
resaleflat_l['remaining_lease_year'] = resaleflat_l['remaining_lease'].str.slice(stop=2).astype('float')
#Substring/slice the month out
resaleflat_l['remaining_lease_month'] = resaleflat_l['remaining_lease'].str.slice(start=9,stop=12)

replacemonth = {}
replacemonth[''] = 0
replacemonth['0 m'] = 0
resaleflat_l['remaining_lease_month'] = resaleflat_l['remaining_lease_month'].replace(replacemonth)

resaleflat_l['remaining_lease_month'] = resaleflat_l['remaining_lease_month'].astype('float') / 12
resaleflat_l['remaining_lease'] = resaleflat_l['remaining_lease_year'] + resaleflat_l['remaining_lease_month']
resaleflat_l = resaleflat_l.drop(['remaining_lease_year','remaining_lease_month'], axis=1)

resaleflat_l.month = resaleflat_l.month.apply(lambda x: float(x[0:4])+float(x[5:7])/12)
#Inpute missing values with remaining lease (take current year-month minus the lease start year)
resaleflat_l.remaining_lease[resaleflat_l.remaining_lease.isnull()] = resaleflat_l['month'] - resaleflat_l['lease_commence_date']

#Don't normalize values as we want to intepret the gradient

#Normalize the month, floor area and remaining lease (timeframe)
# resaleflat_l.month = (resaleflat_l.month - resaleflat_l.month.mean()) / resaleflat_l.month.std()
# resaleflat_l.floor_area_sqm = (resaleflat_l.floor_area_sqm - resaleflat_l.floor_area_sqm.mean()) / resaleflat_l.floor_area_sqm.std()
# resaleflat_l.remaining_lease = (resaleflat_l.remaining_lease - resaleflat_l.remaining_lease.mean()) / resaleflat_l.remaining_lease.std()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [119]:
#Blocks in Singapore is rather irregular and does not impact on the pricing
#Lease commence date is highly correlated with remaining lease as remaining lease = current date - lease commence date
#street name has too many variation, towns would generalize better

resaleflat_l = resaleflat_l.drop(columns=['block','lease_commence_date','street_name','flat_type'])

In [120]:
resaleflat_l.head()

,flat_model,floor_area_sqm,month,remaining_lease,resale_price,storey_range,town
0,Improved,44.0,2017.083333,61.333333,232000.0,10 TO 12,ANG MO KIO
1,New Generation,67.0,2017.083333,60.583333,250000.0,01 TO 03,ANG MO KIO
2,New Generation,67.0,2017.083333,62.416667,262000.0,01 TO 03,ANG MO KIO
3,New Generation,68.0,2017.083333,62.083333,265000.0,04 TO 06,ANG MO KIO
4,New Generation,67.0,2017.083333,62.416667,265000.0,01 TO 03,ANG MO KIO


In [121]:
all_flat_model = pd.get_dummies(resaleflat_l['flat_model'])
all_towns = pd.get_dummies(resaleflat_l['town'])

resaleflat_l = resaleflat_l.drop(columns=['flat_model','town'])
resaleflat_l = pd.concat((resaleflat_l,all_flat_model,all_towns), axis=1)

In [122]:
resaleflat_l.storey_range.unique()

array(['10 TO 12', '01 TO 03', '04 TO 06', '07 TO 09', '13 TO 15',
       '19 TO 21', '22 TO 24', '16 TO 18', '34 TO 36', '28 TO 30',
       '37 TO 39', '49 TO 51', '25 TO 27', '40 TO 42', '31 TO 33',
       '46 TO 48', '43 TO 45', '06 TO 10', '01 TO 05', '11 TO 15',
       '16 TO 20', '21 TO 25', '26 TO 30', '36 TO 40', '31 TO 35'],
      dtype=object)

In [123]:
#Get average of storey range e.g. 01 to 03 would be 2
resaleflat_l.storey_range = resaleflat_l.storey_range.apply(lambda x: (float(x[0:2]) + float(x[7:9]) ) /2 )

#Don't normalize storey as we want to understand the gradient
# resaleflat_l.storey_range = (resaleflat_l.storey_range - resaleflat_l.storey_range.mean()) / resaleflat_l.storey_range.std()

In [124]:
resaleflat_l.head()

,floor_area_sqm,month,remaining_lease,resale_price,storey_range,2-room,Adjoined flat,Apartment,DBSS,Improved,...,PASIR RIS,PUNGGOL,QUEENSTOWN,SEMBAWANG,SENGKANG,SERANGOON,TAMPINES,TOA PAYOH,WOODLANDS,YISHUN
0,44.0,2017.083333,61.333333,232000.0,6.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,67.0,2017.083333,60.583333,250000.0,2.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,67.0,2017.083333,62.416667,262000.0,2.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,68.0,2017.083333,62.083333,265000.0,5.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,67.0,2017.083333,62.416667,265000.0,2.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [125]:
resaleflat_l.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139788 entries, 0 to 37152
Data columns (total 52 columns):
floor_area_sqm            139788 non-null float64
month                     139788 non-null float64
remaining_lease           139788 non-null float64
resale_price              139788 non-null float64
storey_range              139788 non-null float64
2-room                    139788 non-null uint8
Adjoined flat             139788 non-null uint8
Apartment                 139788 non-null uint8
DBSS                      139788 non-null uint8
Improved                  139788 non-null uint8
Improved-Maisonette       139788 non-null uint8
Maisonette                139788 non-null uint8
Model A                   139788 non-null uint8
Model A-Maisonette        139788 non-null uint8
Model A2                  139788 non-null uint8
Multi Generation          139788 non-null uint8
New Generation            139788 non-null uint8
Premium Apartment         139788 non-null uint8
Premium Apartmen

In [126]:
#Convert all to float
# for column in resaleflat_l.columns:
#     resaleflat_l[column] = resaleflat_l[column].astype('float')

In [127]:
#from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [128]:
#Seperate datasets

X = resaleflat_l.drop('resale_price', axis=1)
Y = resaleflat_l['resale_price']

In [129]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Linear Regression

In [130]:
linReg = LinearRegression()
linReg.fit(x_train,y_train)
pred_linReg = linReg.predict(x_test)

In [131]:
pred_linReg

array([286248.42271456, 517112.68523678, 382031.60577034, ...,
       242148.67306877, 328292.88000173, 387658.70216045])

In [132]:
y_test.head()

1040     308000.0
913      460000.0
13013    381000.0
40489    750000.0
10215    480000.0
Name: resale_price, dtype: float64

In [133]:
(y_test - pred_linReg).head()

1040     21751.577285
913     -57112.685237
13013    -1031.605770
40489    13934.137071
10215   -30209.219719
Name: resale_price, dtype: float64

In [134]:
type(pred_linReg)

numpy.ndarray

In [135]:
#Let's see how our model performed

import math

math.sqrt(mean_squared_error(y_test, pred_linReg, multioutput='uniform_average'))

61620.32276426424

In [136]:
#Find RMSE for baseline model - i.e. take average house price

base = math.sqrt(((y_test - y_test.mean()) ** 2).mean())

base

#RMSE of $138415 for prediction of house price using average

138415.3257061527

In [137]:
linear_diff = math.sqrt(((y_test - pred_linReg) ** 2).mean())
linear_diff

#RMSE of $105242 using linear regression

61620.32276426432

In [138]:
def RMSE(actual,predict):
    base = math.sqrt(((y_test - y_test.mean()) ** 2).mean())
    x = math.sqrt(((actual - predict) ** 2).mean())
    return x/base

RMSE(y_test, pred_linReg)

0.44518424856421246

In [139]:
#Get R^2 of prediction, 0.802

linReg.score(x_test,y_test)

0.8018109848303148

In [140]:
y_test.values

array([308000., 460000., 381000., ..., 290000., 310000., 420000.])

In [141]:
pred_linReg

array([286248.42271456, 517112.68523678, 382031.60577034, ...,
       242148.67306877, 328292.88000173, 387658.70216045])

In [142]:
result1 = np.concatenate(([y_test.values], [pred_linReg],[y_test.values - pred_linReg])).T

In [143]:
result1_df = pd.DataFrame(result1, columns=['actual','predict','difference'])

In [144]:
result1_df

,actual,predict,difference
0,308000.0,286248.422715,21751.577285
1,460000.0,517112.685237,-57112.685237
2,381000.0,382031.605770,-1031.605770
3,750000.0,736065.862929,13934.137071
4,480000.0,510209.219719,-30209.219719
5,325000.0,384516.989175,-59516.989175
6,385000.0,325263.149663,59736.850337
7,360000.0,462156.356523,-102156.356523
8,350000.0,397341.718369,-47341.718369
9,337000.0,369939.881733,-32939.881733


In [145]:
print(result1_df['difference'].mean())
print(result1_df['difference'].max())
print(result1_df['difference'].min())

-381.1644469394935
392484.23285748065
-488275.0150200464


In [146]:
result1_df[result1_df['difference'] > 100000]

,actual,predict,difference
26,688888.0,570506.707406,118381.292594
36,800000.0,661246.875804,138753.124196
92,710000.0,530539.283599,179460.716401
103,818000.0,527623.860173,290376.139827
122,850000.0,662579.406716,187420.593284
128,888000.0,764935.769623,123064.230377
133,800000.0,547494.974051,252505.025949
177,830000.0,655007.574834,174992.425166
182,920000.0,704114.881362,215885.118638
187,910000.0,706078.216376,203921.783624


In [147]:
result1_df[result1_df['difference'] < -100000]

,actual,predict,difference
7,360000.0,462156.356523,-102156.356523
55,373000.0,484950.040196,-111950.040196
75,450000.0,568715.317772,-118715.317772
83,428000.0,547064.206831,-119064.206831
162,488000.0,598646.243199,-110646.243199
224,303888.0,437990.990829,-134102.990829
227,523000.0,685116.500663,-162116.500663
244,537000.0,684917.869942,-147917.869942
250,433000.0,533889.755255,-100889.755255
259,295000.0,428803.363226,-133803.363226


In [148]:
['Intercept',linReg.intercept_]

['Intercept', 21223974.89753688]

In [154]:
gradient = pd.DataFrame(np.concatenate(([X.columns],[linReg.coef_]),axis=0).T, columns=['Variable','Gradient'])

intercept = pd.DataFrame(['Intercept',linReg.intercept_]).transpose()
intercept.columns = ['Variable','Gradient']

meanprice = pd.DataFrame(['Mean Price',resaleflat['resale_price'].mean()]).transpose()
meanprice.columns = ['Variable','Gradient']

medianprice = pd.DataFrame(['Median Price',resaleflat['resale_price'].median()]).transpose()
medianprice.columns = ['Variable','Gradient']

gradient = pd.concat((meanprice,medianprice,gradient,intercept),axis=0)
print(linReg.intercept_) #Intercept is 21million but it has to minus a lot of gradient for features not there
print(resaleflat['resale_price'].mean()) #Mean price of house is $447,302
print(resaleflat['resale_price'].median()) #Median price of house is $420,000
gradient

21223974.89753688
447302.2250540101
420000.0


,Variable,Gradient
0,Mean Price,447302
0,Median Price,420000
0,floor_area_sqm,4414.91
1,month,-10496.1
2,remaining_lease,204.031
3,storey_range,9049.49
4,2-room,9193.4
5,Adjoined flat,-80398.3
6,Apartment,-68090.3
7,DBSS,156085


In [155]:
#Predict price of flat in Serangoon

serangoonpredict = pd.DataFrame(np.zeros((1,51), dtype=float, order='C'),columns=x_test.columns)
#Serangoon 5 room flat, 1302 sqft to 121 sqm
serangoonpredict['floor_area_sqm'] = 121
#serangoonpredict['lease_commence_date'] = 1989
serangoonpredict['remaining_lease'] = 99 - (2019-1989)
serangoonpredict['storey_range'] = 5
serangoonpredict['Improved'] = 1
serangoonpredict['SERANGOON'] = 1
serangoonpredict['month'] = 2019

linReg.predict(serangoonpredict)

array([539020.69915858])